In [95]:
import os
from google.cloud import bigquery
from google.cloud import storage

project_id = "amazon-product-reviews-452322"
region = "us-central1"
bq_client = bigquery.Client(project=project_id, location=region)
staging_dataset_id = f"{project_id}.product_data_stg"

staging_dataset = bigquery.Dataset(staging_dataset_id)
staging_dataset.location = region

staging_dataset = bq_client.create_dataset(staging_dataset, exists_ok=True)
print(f"Created or verified existence of dataset: {staging_dataset.dataset_id}")


Created or verified existence of dataset: product_data_stg


In [96]:
%%bigquery --project amazon-product-reviews-452322
SELECT table_name
FROM  `amazon-product-reviews-452322`.`product_data_raw`.INFORMATION_SCHEMA.TABLES;


Query is running:   0%|          |

Downloading:   0%|          |

,table_name
0,reviews_Automotive
1,reviews_Books
2,product_meta_data_2
3,Products_Meta_Movies_And_Tv
4,sales
5,Products_Meta_Books
6,Products_Meta_Digital_Music
7,product-image-data-all
8,user
9,reviews_Digital_Music


# Requirement 5

## Staging table: membership_info

In [99]:
%%bigquery --project amazon-product-reviews-452322
SELECT *
FROM product_data_raw.amazon_prime_users
LIMIT 5;

Query is running:   0%|          |

Downloading:   0%|          |

,user_id,name,email_address,username,date_of_birth,gender,location,membership_start_date,membership_end_date,subscription_plan,payment_information,renewal_status,usage_frequency,purchase_history,favorite_genres,devices_used,engagement_metrics,feedback_ratings,customer_support_interactions,_data_source,_load_time
0,730,Deborah Owens,smithdouglas@example.net,smithdouglas,1971-12-24,Male,Williamsfort,2024-01-01,2024-12-31,Annual,Amex,Auto-renew,Frequent,Books,Sci-Fi,Smart TV,Low,3.2,3,initial_loads/prime_users,2025-03-01 23:25:54.058916+00:00
1,1027,Corey Riley,michaeljohnson@example.net,michaeljohnson,1947-07-30,Male,West Lisaport,2024-01-01,2024-12-31,Monthly,Visa,Auto-renew,Frequent,Books,Comedy,Smartphone,High,4.9,2,initial_loads/prime_users,2025-03-01 23:25:54.058916+00:00
2,1083,Karla Lopez,michaellarson@example.com,michaellarson,1980-05-05,Male,North Tammy,2024-01-01,2024-12-31,Annual,Visa,Manual,Frequent,Clothing,Documentary,Smart TV,Low,3.9,8,initial_loads/prime_users,2025-03-01 23:25:54.058916+00:00
3,1277,Melanie Martin,riceallison@example.com,riceallison,1969-01-16,Male,Simmonsside,2024-01-01,2024-12-31,Annual,Mastercard,Auto-renew,Frequent,Clothing,Romance,Tablet,Low,4.9,3,initial_loads/prime_users,2025-03-01 23:25:54.058916+00:00
4,1546,Charles Haney,jamescaldwell@example.org,jamescaldwell,1935-02-19,Male,West Johnathanview,2024-01-01,2024-12-31,Annual,Mastercard,Auto-renew,Frequent,Clothing,Documentary,Smart TV,Low,4.3,3,initial_loads/prime_users,2025-03-01 23:25:54.058916+00:00


In [100]:
!pip install --quiet google-cloud-bigquery

from google.cloud import bigquery
import pandas as pd

project_id = "amazon-product-reviews-452322"
bq_client = bigquery.Client(project=project_id)

query = """
SELECT *
FROM `amazon-product-reviews-452322.product_data_raw.amazon_prime_users`
LIMIT 100
"""
df = bq_client.query(query).to_dataframe()

pd.set_option('display.max_columns', None)
pd.set_option('display.max_colwidth', None)
pd.set_option('display.width', 1000)


df.head(10)


,user_id,name,email_address,username,date_of_birth,gender,location,membership_start_date,membership_end_date,subscription_plan,payment_information,renewal_status,usage_frequency,purchase_history,favorite_genres,devices_used,engagement_metrics,feedback_ratings,customer_support_interactions,_data_source,_load_time
0,730,Deborah Owens,smithdouglas@example.net,smithdouglas,1971-12-24,Male,Williamsfort,2024-01-01,2024-12-31,Annual,Amex,Auto-renew,Frequent,Books,Sci-Fi,Smart TV,Low,3.2,3,initial_loads/prime_users,2025-03-01 23:25:54.058916+00:00
1,1027,Corey Riley,michaeljohnson@example.net,michaeljohnson,1947-07-30,Male,West Lisaport,2024-01-01,2024-12-31,Monthly,Visa,Auto-renew,Frequent,Books,Comedy,Smartphone,High,4.9,2,initial_loads/prime_users,2025-03-01 23:25:54.058916+00:00
2,1083,Karla Lopez,michaellarson@example.com,michaellarson,1980-05-05,Male,North Tammy,2024-01-01,2024-12-31,Annual,Visa,Manual,Frequent,Clothing,Documentary,Smart TV,Low,3.9,8,initial_loads/prime_users,2025-03-01 23:25:54.058916+00:00
3,1277,Melanie Martin,riceallison@example.com,riceallison,1969-01-16,Male,Simmonsside,2024-01-01,2024-12-31,Annual,Mastercard,Auto-renew,Frequent,Clothing,Romance,Tablet,Low,4.9,3,initial_loads/prime_users,2025-03-01 23:25:54.058916+00:00
4,1546,Charles Haney,jamescaldwell@example.org,jamescaldwell,1935-02-19,Male,West Johnathanview,2024-01-01,2024-12-31,Annual,Mastercard,Auto-renew,Frequent,Clothing,Documentary,Smart TV,Low,4.3,3,initial_loads/prime_users,2025-03-01 23:25:54.058916+00:00
5,1593,Derek Cooper,whayes@example.com,whayes,1941-12-23,Female,South Lauraborough,2024-01-01,2024-12-31,Annual,Visa,Manual,Frequent,Electronics,Comedy,Smart TV,High,4.3,5,initial_loads/prime_users,2025-03-01 23:25:54.058916+00:00
6,1877,Julia Chambers,karenrodriguez@example.org,karenrodriguez,2003-06-21,Female,Garrettland,2024-01-01,2024-12-31,Annual,Visa,Manual,Frequent,Electronics,Action,Smart TV,High,4.3,2,initial_loads/prime_users,2025-03-01 23:25:54.058916+00:00
7,2134,Lauren Stevenson,laurenmyers@example.com,laurenmyers,1934-05-27,Male,Johnsonland,2024-01-01,2024-12-31,Monthly,Visa,Auto-renew,Frequent,Clothing,Romance,Smartphone,Medium,4.5,8,initial_loads/prime_users,2025-03-01 23:25:54.058916+00:00
8,2334,Deborah Brown,cooleylisa@example.net,cooleylisa,1990-05-10,Female,North Hollymouth,2024-01-01,2024-12-31,Monthly,Amex,Manual,Frequent,Clothing,Drama,Smartphone,High,3.5,8,initial_loads/prime_users,2025-03-01 23:25:54.058916+00:00
9,2346,James Mcgee MD,kellylane@example.net,kellylane,1939-02-02,Male,Elizabethside,2024-01-01,2024-12-31,Annual,Mastercard,Manual,Frequent,Books,Horror,Smartphone,Medium,3.4,1,initial_loads/prime_users,2025-03-01 23:25:54.058916+00:00


In [101]:
%%bigquery --project amazon-product-reviews-452322
SELECT column_name, data_type
FROM `amazon-product-reviews-452322.product_data_raw.INFORMATION_SCHEMA.COLUMNS`
WHERE table_name = 'amazon_prime_users'

Query is running:   0%|          |

Downloading:   0%|          |

,column_name,data_type
0,user_id,STRING
1,name,STRING
2,email_address,STRING
3,username,STRING
4,date_of_birth,STRING
5,gender,STRING
6,location,STRING
7,membership_start_date,STRING
8,membership_end_date,STRING
9,subscription_plan,STRING


In [102]:
%%bigquery --project amazon-product-reviews-452322
CREATE OR REPLACE TABLE `amazon-product-reviews-452322.product_data_stg.prime_users` AS
SELECT
    CAST(user_id AS INT64) AS user_id,
    name,
    email_address,
    username,
    CAST(date_of_birth AS DATE) AS date_of_birth,
    gender,
    location,
    CAST(membership_start_date AS DATE) AS membership_start_date,
    CAST(membership_end_date AS DATE) AS membership_end_date,
    subscription_plan,
    CAST(payment_information AS STRING) AS payment_information,
    renewal_status,
    usage_frequency,
    purchase_history,
    favorite_genres,
    devices_used,
    engagement_metrics,
    CAST(feedback_ratings AS FLOAT64) AS feedback_ratings,
    CAST(customer_support_interactions AS INT64) AS customer_support_interactions,
    _data_source,
    _load_time
FROM `amazon-product-reviews-452322.product_data_raw.amazon_prime_users`;


Query is running:   0%|          |

""


In [103]:
%%bigquery --project amazon-product-reviews-452322
SELECT column_name, data_type
FROM `amazon-product-reviews-452322.product_data_stg.INFORMATION_SCHEMA.COLUMNS`
WHERE table_name = 'prime_users';


Query is running:   0%|          |

Downloading:   0%|          |

,column_name,data_type
0,user_id,INT64
1,name,STRING
2,email_address,STRING
3,username,STRING
4,date_of_birth,DATE
5,gender,STRING
6,location,STRING
7,membership_start_date,DATE
8,membership_end_date,DATE
9,subscription_plan,STRING


# Requirement 6

## Staging table: Products_Meta_Gift_Cards





In [104]:
%%bigquery --project amazon-product-reviews-452322
SELECT *
FROM product_data_raw.Products_Meta_Gift_Cards
LIMIT 3;

Query is running:   0%|          |

Downloading:   0%|          |

,main_category,title,average_rating,rating_number,features,description,price,images,videos,store,categories,details,parent_asin,bought_together,_data_source,_load_time
0,None,$400 Electronic Airline Voucher,1.0,1,[],[],NaN,"[{""thumb"": ""https://m.media-amazon.com/images/I/51qU-l-Be4L._AC_US40_.jpg"", ""large"": ""https://m.media-amazon.com/images/I/51qU-l-Be4L._AC_.jpg"", ""variant"": ""MAIN"", ""hi_res"": ""https://m.media-amazon.com/images/I/91JywPH-0-L._AC_SL1500_.jpg""}]",[],United Airlines,"[""Gift Cards"", ""Gift Card Categories"", ""Specialty Cards""]","{""Item Weight"": ""0.247 ounces"", ""Manufacturer"": ""United Airlines"", ""Date First Available"": ""April 25, 2013""}",B00CJ0WVYC,null,initial_loads/product_meta_data,2025-03-01 21:55:57.411206+00:00
1,Gift Cards,2 Hours of Moving Services with Truck By Trade Movers,1.0,1,"[""Instant 50% off"", ""2 Hours of full service moving with truck"", ""2 Professional movers"", ""16 Foot moving truck included"", ""Moving supplies and equipment included(no boxes)""]","[""With over 8,000 fans on Facebook, it's clear that Trade Movers has made a great impression on Chicagoland residents looking for an affordable and efficient move. This card will provide you 2 hours of moving services with Trade Movers in New York and Chicago locations. Details on card is as advertised and will be serviced upon usage on the day of the move. Additional hours must be paid in cash, cashiers check, or money order Buy this gift card for a friend, family or coworker, as we see it no body likes to move. This gift card is good to use towards products and services offered by Trade Movers. Purchasing this gift card will get you two movers for 2 hours with truck. Supplies include: Shrink wrap Padding tape Equipment: Bins Dollies tools""]",NaN,"[{""thumb"": ""https://m.media-amazon.com/images/I/41eRtDdAb9L._SX38_SY50_CR,0,0,38,50_.jpg"", ""large"": ""https://m.media-amazon.com/images/I/41eRtDdAb9L.jpg"", ""variant"": ""MAIN"", ""hi_res"": null}, {""thumb"": ""https://m.media-amazon.com/images/I/51t+0ADxGDL._SX38_SY50_CR,0,0,38,50_.jpg"", ""large"": ""https://m.media-amazon.com/images/I/51t+0ADxGDL.jpg"", ""variant"": ""PT01"", ""hi_res"": null}, {""thumb"": ""https://m.media-amazon.com/images/I/514gaFH7aCL._SX38_SY50_CR,0,0,38,50_.jpg"", ""large"": ""https://m.media-amazon.com/images/I/514gaFH7aCL.jpg"", ""variant"": ""PT03"", ""hi_res"": ""https://m.media-amazon.com/images/I/81v+L1TiZ9L._SL1500_.jpg""}]",[],TradeMovers,"[""Gift Cards"", ""Gift Card Categories"", ""Specialty Cards""]","{""Date First Available"": ""August 30, 2013""}",B00EW4LFKO,null,initial_loads/product_meta_data,2025-03-01 21:55:57.411206+00:00
2,None,Amazing Clubs Gift Card 1000,1.0,3,"[""Gift of the Month Clubs are the perfect gift when you want to say you care not just once, but month after month!"", ""We offer dozens of gourmet gift clubs for every person, every passion and every pocketbook."", ""We're rated #1 at what we do and with over 1 Million satisfied customers you can relax knowing that your gift will always be well received."", ""Selection, Quality and Value are what set us apart from our competitors."", ""At Amazing Clubs we offer the largest selection, highest quality and best value of any gift club provider.""]",[],NaN,"[{""thumb"": ""https://m.media-amazon.com/images/I/4178DB8J5fL._SX38_SY50_CR,0,0,38,50_.jpg"", ""large"": ""https://m.media-amazon.com/images/I/4178DB8J5fL.jpg"", ""variant"": ""MAIN"", ""hi_res"": null}]",[],Amazing Clubs,"[""Gift Cards"", ""Gift Card Categories"", ""Clothing, Shoes & Accessories""]",{},B00A7C9BOW,null,initial_loads/product_meta_data,2025-03-01 21:55:57.411206+00:00


In [105]:
%%bigquery --project amazon-product-reviews-452322
CREATE OR REPLACE TABLE `amazon-product-reviews-452322.product_data_stg.product_meta_Gift_Cards` AS
SELECT
    main_category,
    title,
    CAST(average_rating AS FLOAT64) AS average_rating,
    CAST(rating_number AS INT64) AS rating_number,
    NULLIF(features, '[]') AS features,
    NULLIF(description, '[]') AS description,
    CAST(NULLIF(CAST(price AS STRING), '') AS FLOAT64) AS price,
    REGEXP_REPLACE(store, r'[^a-zA-Z0-9\s]', '') AS store,
    JSON_QUERY_ARRAY(images) AS images,
    JSON_QUERY_ARRAY(videos) AS videos,
    JSON_QUERY_ARRAY(categories) AS categories,
    details,
    parent_asin,
    NULLIF(bought_together, 'null') AS bought_together,
    _data_source,
    _load_time
FROM `amazon-product-reviews-452322.product_data_raw.Products_Meta_Gift_Cards`;


Query is running:   0%|          |

""


In [106]:
%%bigquery --project amazon-product-reviews-452322
SELECT column_name, data_type
FROM `amazon-product-reviews-452322.product_data_stg.INFORMATION_SCHEMA.COLUMNS`
WHERE table_name = 'product_meta_Gift_Cards';


Query is running:   0%|          |

Downloading:   0%|          |

,column_name,data_type
0,main_category,STRING
1,title,STRING
2,average_rating,FLOAT64
3,rating_number,INT64
4,features,STRING
5,description,STRING
6,price,FLOAT64
7,store,STRING
8,images,ARRAY<STRING>
9,videos,ARRAY<STRING>


In [107]:
%%bigquery --project amazon-product-reviews-452322
SELECT *
FROM `amazon-product-reviews-452322.product_data_stg.product_meta_Gift_Cards`
LIMIT 3;

Query is running:   0%|          |

Downloading:   0%|          |

,main_category,title,average_rating,rating_number,features,description,price,store,images,videos,categories,details,parent_asin,bought_together,_data_source,_load_time
0,Gift Cards,Amazon.com Gift Card,3.0,9,None,None,NaN,None,"[{""thumb"":""https://m.media-amazon.com/images/I/01RmK+J4pJL._SX38_SY50_CR,0,0,38,50_.gif"",""large"":""https://m.media-amazon.com/images/I/01RmK+J4pJL.gif"",""variant"":""MAIN"",""hi_res"":null}]",[],[],"{""Is Discontinued By Manufacturer"": ""No"", ""Date First Available"": ""December 12, 2003""}",B00067G16E,None,initial_loads/product_meta_data,2025-03-01 21:49:10.834829+00:00
1,Gift Cards,Amazon.com Gift Card,3.0,9,None,None,NaN,None,"[{""thumb"":""https://m.media-amazon.com/images/I/01RmK+J4pJL._SX38_SY50_CR,0,0,38,50_.gif"",""large"":""https://m.media-amazon.com/images/I/01RmK+J4pJL.gif"",""variant"":""MAIN"",""hi_res"":null}]",[],[],"{""Is Discontinued By Manufacturer"": ""No"", ""Date First Available"": ""December 12, 2003""}",B00067G16E,None,initial_loads/product_meta_data,2025-03-01 23:26:54.742358+00:00
2,Gift Cards,Amazon.com Gift Card,3.0,9,None,None,NaN,None,"[{""thumb"":""https://m.media-amazon.com/images/I/01RmK+J4pJL._SX38_SY50_CR,0,0,38,50_.gif"",""large"":""https://m.media-amazon.com/images/I/01RmK+J4pJL.gif"",""variant"":""MAIN"",""hi_res"":null}]",[],[],"{""Is Discontinued By Manufacturer"": ""No"", ""Date First Available"": ""December 12, 2003""}",B00067G16E,None,initial_loads/product_meta_data,2025-03-01 21:58:46.558460+00:00


## For all other product meta data:

- Applicances
- Automotive
- Books
- Digital Music
- Magazine Subscriptions
- Movies and TV

In [108]:
%%bigquery --project amazon-product-reviews-452322
-- Create staging table for Appliances
CREATE OR REPLACE TABLE `amazon-product-reviews-452322.product_data_stg.product_meta_Appliances` AS
SELECT
  main_category,
  title,
  CAST(average_rating AS FLOAT64) AS average_rating,
  CAST(rating_number AS INT64) AS rating_number,
  NULLIF(features, '[]') AS features,
  NULLIF(description, '[]') AS description,
  CAST(NULLIF(CAST(price AS STRING), '') AS FLOAT64) AS price,
  REGEXP_REPLACE(store, r'[^a-zA-Z0-9\s]', '') AS store,
  JSON_QUERY_ARRAY(images) AS images,
  JSON_QUERY_ARRAY(videos) AS videos,
  JSON_QUERY_ARRAY(categories) AS categories,
  details,
  parent_asin,
  NULLIF(bought_together, 'null') AS bought_together,
  _data_source,
  _load_time
FROM `amazon-product-reviews-452322.product_data_raw.Products_Meta_Appliances`;

-- Create staging table for Automotive
CREATE OR REPLACE TABLE `amazon-product-reviews-452322.product_data_stg.product_meta_Automotive` AS
SELECT
  main_category,
  title,
  CAST(average_rating AS FLOAT64) AS average_rating,
  CAST(rating_number AS INT64) AS rating_number,
  NULLIF(features, '[]') AS features,
  NULLIF(description, '[]') AS description,
  CAST(NULLIF(CAST(price AS STRING), '') AS FLOAT64) AS price,
  REGEXP_REPLACE(store, r'[^a-zA-Z0-9\s]', '') AS store,
  JSON_QUERY_ARRAY(images) AS images,
  JSON_QUERY_ARRAY(videos) AS videos,
  JSON_QUERY_ARRAY(categories) AS categories,
  details,
  parent_asin,
  NULLIF(bought_together, 'null') AS bought_together,
  _data_source,
  _load_time
FROM `amazon-product-reviews-452322.product_data_raw.Products_Meta_Automotive`;

-- Create staging table for Books
CREATE OR REPLACE TABLE `amazon-product-reviews-452322.product_data_stg.product_meta_Books` AS
SELECT
  main_category,
  title,
  CAST(average_rating AS FLOAT64) AS average_rating,
  CAST(rating_number AS INT64) AS rating_number,
  NULLIF(features, '[]') AS features,
  NULLIF(description, '[]') AS description,
  CAST(NULLIF(CAST(price AS STRING), '') AS FLOAT64) AS price,
  REGEXP_REPLACE(store, r'[^a-zA-Z0-9\s]', '') AS store,
  JSON_QUERY_ARRAY(images) AS images,
  JSON_QUERY_ARRAY(videos) AS videos,
  JSON_QUERY_ARRAY(categories) AS categories,
  details,
  parent_asin,
  NULLIF(bought_together, 'null') AS bought_together,
  _data_source,
  _load_time
FROM `amazon-product-reviews-452322.product_data_raw.Products_Meta_Books`;

-- Create staging table for Digital Music
CREATE OR REPLACE TABLE `amazon-product-reviews-452322.product_data_stg.product_meta_Digital_Music` AS
SELECT
  main_category,
  title,
  CAST(average_rating AS FLOAT64) AS average_rating,
  CAST(rating_number AS INT64) AS rating_number,
  NULLIF(features, '[]') AS features,
  NULLIF(description, '[]') AS description,
  CAST(NULLIF(CAST(price AS STRING), '') AS FLOAT64) AS price,
  REGEXP_REPLACE(store, r'[^a-zA-Z0-9\s]', '') AS store,
  JSON_QUERY_ARRAY(images) AS images,
  JSON_QUERY_ARRAY(videos) AS videos,
  JSON_QUERY_ARRAY(categories) AS categories,
  details,
  parent_asin,
  NULLIF(bought_together, 'null') AS bought_together,
  _data_source,
  _load_time
FROM `amazon-product-reviews-452322.product_data_raw.Products_Meta_Digital_Music`;

-- Create staging table for Gift Cards
CREATE OR REPLACE TABLE `amazon-product-reviews-452322.product_data_stg.product_meta_Gift_Cards` AS
SELECT
  main_category,
  title,
  CAST(average_rating AS FLOAT64) AS average_rating,
  CAST(rating_number AS INT64) AS rating_number,
  NULLIF(features, '[]') AS features,
  NULLIF(description, '[]') AS description,
  CAST(NULLIF(CAST(price AS STRING), '') AS FLOAT64) AS price,
  REGEXP_REPLACE(store, r'[^a-zA-Z0-9\s]', '') AS store,
  JSON_QUERY_ARRAY(images) AS images,
  JSON_QUERY_ARRAY(videos) AS videos,
  JSON_QUERY_ARRAY(categories) AS categories,
  details,
  parent_asin,
  NULLIF(bought_together, 'null') AS bought_together,
  _data_source,
  _load_time
FROM `amazon-product-reviews-452322.product_data_raw.Products_Meta_Gift_Cards`;

-- Create staging table for Magazine Subscriptions
CREATE OR REPLACE TABLE `amazon-product-reviews-452322.product_data_stg.product_meta_Magazine_Subscriptions` AS
SELECT
  main_category,
  title,
  CAST(average_rating AS FLOAT64) AS average_rating,
  CAST(rating_number AS INT64) AS rating_number,
  NULLIF(features, '[]') AS features,
  NULLIF(description, '[]') AS description,
  CAST(NULLIF(CAST(price AS STRING), '') AS FLOAT64) AS price,
  REGEXP_REPLACE(store, r'[^a-zA-Z0-9\s]', '') AS store,
  JSON_QUERY_ARRAY(images) AS images,
  JSON_QUERY_ARRAY(videos) AS videos,
  JSON_QUERY_ARRAY(categories) AS categories,
  details,
  parent_asin,
  NULLIF(bought_together, 'null') AS bought_together,
  _data_source,
  _load_time
FROM `amazon-product-reviews-452322.product_data_raw.Products_Meta_Magazine_Subscriptions`;

-- Create staging table for Movies and TV
CREATE OR REPLACE TABLE `amazon-product-reviews-452322.product_data_stg.product_meta_Movies_And_Tv` AS
SELECT
  main_category,
  title,
  CAST(average_rating AS FLOAT64) AS average_rating,
  CAST(rating_number AS INT64) AS rating_number,
  NULLIF(features, '[]') AS features,
  NULLIF(description, '[]') AS description,
  CAST(NULLIF(CAST(price AS STRING), '') AS FLOAT64) AS price,
  REGEXP_REPLACE(store, r'[^a-zA-Z0-9\s]', '') AS store,
  JSON_QUERY_ARRAY(images) AS images,
  JSON_QUERY_ARRAY(videos) AS videos,
  JSON_QUERY_ARRAY(categories) AS categories,
  details,
  parent_asin,
  NULLIF(bought_together, 'null') AS bought_together,
  _data_source,
  _load_time
FROM `amazon-product-reviews-452322.product_data_raw.Products_Meta_Movies_And_Tv`;



Query is running:   0%|          |

""


In [109]:
%%bigquery --project amazon-product-reviews-452322

-- Create combined table with all product categories
CREATE OR REPLACE TABLE `amazon-product-reviews-452322.product_data_stg.product_meta_data` AS
SELECT * FROM `amazon-product-reviews-452322.product_data_stg.product_meta_Appliances`
UNION ALL
SELECT * FROM `amazon-product-reviews-452322.product_data_stg.product_meta_Automotive`
UNION ALL
SELECT * FROM `amazon-product-reviews-452322.product_data_stg.product_meta_Books`
UNION ALL
SELECT * FROM `amazon-product-reviews-452322.product_data_stg.product_meta_Digital_Music`
UNION ALL
SELECT * FROM `amazon-product-reviews-452322.product_data_stg.product_meta_Gift_Cards`
UNION ALL
SELECT * FROM `amazon-product-reviews-452322.product_data_stg.product_meta_Magazine_Subscriptions`
UNION ALL
SELECT * FROM `amazon-product-reviews-452322.product_data_stg.product_meta_Movies_And_Tv`;

Query is running:   0%|          |

""


In [146]:
%%bigquery --project amazon-product-reviews-452322
DROP TABLE IF EXISTS `amazon-product-reviews-452322.product_data_stg.product_meta_Appliances`;
DROP TABLE IF EXISTS `amazon-product-reviews-452322.product_data_stg.product_meta_Automotive`;
DROP TABLE IF EXISTS `amazon-product-reviews-452322.product_data_stg.product_meta_Books`;
DROP TABLE IF EXISTS `amazon-product-reviews-452322.product_data_stg.product_meta_Digital_Music`;
DROP TABLE IF EXISTS `amazon-product-reviews-452322.product_data_stg.product_meta_Gift_Cards`;
DROP TABLE IF EXISTS `amazon-product-reviews-452322.product_data_stg.product_meta_Magazine_Subscriptions`;
DROP TABLE IF EXISTS `amazon-product-reviews-452322.product_data_stg.product_meta_Movies_And_Tv`;

Query is running:   0%|          |

""


## Staging table: reviews_Gift_Cards





In [110]:
%%bigquery --project amazon-product-reviews-452322
SELECT *
FROM `amazon-product-reviews-452322.product_data_raw.reviews_Gift_Cards`
LIMIT 3;

Query is running:   0%|          |

Downloading:   0%|          |

,rating,title,text,images,asin,parent_asin,user_id,timestamp,helpful_vote,verified_purchase,details,videos,_data_source,_load_time
0,1.0,Amazon has lost it.,"I bought an ""instant"" gift certificate for my sister. The kind I print myself then send.<br /><br />""How bad can it be?"" I thought.<br />""When I pay-at-the-pump at the gas station the credit card transaction takes about 4.5 seconds.""<br /><br />Not so with Amazon's gift cards. Pretty retarded if you think about it. What do we have the web for anyway?<br /><br />Funny thing is, Amazon has patents on providing good web shopping experiences for their customers. Too bad ""inventing"" is different than ""doing"". They really don't seem to care any more.",[],B00067G16E,B00067G16E,AGPVL6NHGFHKSPKWGEPDDC426R2Q,1217986106000,3,False,{},[],initial_loads/product_reviews,2025-03-01 23:27:44.941421+00:00
1,1.0,Don't Bother,"I ordered the card as a Christmas gift and I found cheap paper instead of plastic when it arrived. The ""card"" was worn like the spine of a used paperback. I was on the phone with customer service 18 minutes to cancel the order and find the locations where the plastic version is sold. I think I'll buy a different gift entirely.[[ASIN:B001GXRQW0 Amazon.com $25 Gift Card (0108)]]",[],B00067G16E,B00067G16E,AGCW2FCT4OB7BN3E7QMB63RYEA2A,1229656714000,2,False,{},[],initial_loads/product_reviews,2025-03-01 23:27:44.941421+00:00
2,1.0,Pointless and frustrating!!,"I wanted an item that was 300.00 after tax and shipping was free and was given 3 different gift cards for 100.00 each for the 300.00 item I wanted!<br /><br />I have my credit card on my account from a past order but wasn't prepared to use it for the item I wanted.... I checked the box saying I wanted to use the 300.00 gift card balance option it gave me and since the gift cards were 100.00 each, they used 1 of the 100.00 cards and charged the rest to my unprepared cedit card... I was shocked!<br /><br />I wasnt prepared for that with my credit card especially after the holidays, so of course it declined the extra 200.00 I was expecting the gift cards to have covered....<br />and I wanted to use the 300.00 in gift cards, not my credit card on my account from previous use of Amazon.<br /><br />I called customer service and kept getting a woman who I could not understand, telling me the same thing over and over as if she was reading from a script to me, telling me she couldnt help me and would forward my issue to another department and they would get back with me by email within 12 hours.<br /><br />I get an email later stating I need to choose another payment method, as my credit card declined! UM, DUH! I didnt wish to use my card, I wanted to use the gift card balance it showed I had of 300.00!!<br /><br />So, I wrote to explain this back to them and no answer, it has been a week, and my order was cancelled.<br /><br />I still have a 300.00 gift card balance but cant use it for the 300.00 item I wanted only can use 100.00 at a time! RETARDED!<br /><br />This is the 3rd issue with amazon gift cards I have encountered!",[],B00067G16E,B00067G16E,AHNENMYIF5JOHFCMTRNBCKFIF6XQ,1231190590000,2,False,{},[],initial_loads/product_reviews,2025-03-01 23:27:44.941421+00:00


In [111]:
%%bigquery --project amazon-product-reviews-452322
SELECT column_name, data_type
FROM `amazon-product-reviews-452322.product_data_raw.INFORMATION_SCHEMA.COLUMNS`
WHERE table_name = 'reviews_Gift_Cards';


Query is running:   0%|          |

Downloading:   0%|          |

,column_name,data_type
0,rating,FLOAT64
1,title,STRING
2,text,STRING
3,images,ARRAY<STRING>
4,asin,STRING
5,parent_asin,STRING
6,user_id,STRING
7,timestamp,INT64
8,helpful_vote,INT64
9,verified_purchase,BOOL


In [112]:
%%bigquery --project amazon-product-reviews-452322
CREATE OR REPLACE TABLE `amazon-product-reviews-452322.product_data_stg.reviews_Gift_Cards` AS
SELECT
    CAST(rating AS FLOAT64) AS rating,
    title,
    text,
    images,
    asin,
    parent_asin,
    user_id,
    DATE(TIMESTAMP_MILLIS(CAST(timestamp AS INT64))) AS review_date,
    CAST(helpful_vote AS INT64) AS helpful_vote,
    CAST(verified_purchase AS BOOL) AS verified_purchase,
    details,
    videos,
    _data_source,
    _load_time
FROM `amazon-product-reviews-452322.product_data_raw.reviews_Gift_Cards`;


Query is running:   0%|          |

""


In [113]:
%%bigquery --project amazon-product-reviews-452322
SELECT column_name, data_type
FROM `amazon-product-reviews-452322.product_data_stg.INFORMATION_SCHEMA.COLUMNS`
WHERE table_name = 'reviews_Gift_Cards';


Query is running:   0%|          |

Downloading:   0%|          |

,column_name,data_type
0,rating,FLOAT64
1,title,STRING
2,text,STRING
3,images,ARRAY<STRING>
4,asin,STRING
5,parent_asin,STRING
6,user_id,STRING
7,review_date,DATE
8,helpful_vote,INT64
9,verified_purchase,BOOL


In [114]:
%%bigquery --project amazon-product-reviews-452322
SELECT *
FROM `amazon-product-reviews-452322.product_data_stg.reviews_Gift_Cards`
LIMIT 3;

Query is running:   0%|          |

Downloading:   0%|          |

,rating,title,text,images,asin,parent_asin,user_id,review_date,helpful_vote,verified_purchase,details,videos,_data_source,_load_time
0,1.0,Amazon.com gift card delivered but never received (stolen?),"U.S.P.S delivery tracking claims that my gift card was delivered, but I never got it. This appears to be the first item ever stolen from my mailbox in 7 1/2 years. I suspect that Amazon puts ""Amazon.com"" as the return address, thereby alerting EVERYONE that it contains a gift card, which of couse ANYONE can use.<br /><br />The upshot is that I'm out the $25.00 I paid to Amazon, and Amazon provides no way for me to even inform them that the gift card was stolen/lost.<br />My recommendation: Don't buy Amazon gift cards. They are sitting ducks.<br />Instead, give a check or some other type of gift card for that you buy in person.",[],B001GXRQW0,B001GXRQW0,AGKKPX2CMD7B6OCV6OMXGTHK4IGA,2010-12-19,2,False,{},[],initial_loads/product_reviews,2025-03-01 23:01:14.370094+00:00
1,1.0,"AVOID, does not work on amazon.co.uk","I bougt an amazon.com gift card for a friend living in Europe.<br /><br />He was unable to use it on amazon.co.uk, meaning he has to place the order on amazon.com and pay additional importing fees.",[],B001H324JO,B001H324JO,AGZQWFA5GH5B4XWPTY3X2NSQTIIQ,2010-10-03,2,False,{},[],initial_loads/product_reviews,2025-03-01 23:01:14.370094+00:00
2,1.0,GIFT CODE REDEEMED IN WRONG AND UNAUTHORIZED ACCOUNT.,"SRES. AMAZON BUY A GIFT CARD FOR USE UPON and I could not use it because it was redeemed in another account without my authorization. NO CLAIM Use code GIFT CARD IN THE AMOUNT OF $ 100. I need my money back or tell me who use my code gives card improperly, that is a crime. Thank you.",[],B001H53QDU,B001H53QDU,AFPF5B6GITOYV6MUYJDVACXWCDMA,2013-03-26,1,True,{},[],initial_loads/product_reviews,2025-03-01 23:01:14.370094+00:00


## For all other product review data:

- Applicances
- Automotive
- Books
- Digital Music
- Magazine Subscriptions
- Movies and TV

In [115]:
%%bigquery --project amazon-product-reviews-452322
-- Create staging table for Appliances reviews
CREATE OR REPLACE TABLE `amazon-product-reviews-452322.product_data_stg.reviews_Appliances` AS
SELECT
    CAST(rating AS FLOAT64) AS rating,
    title,
    text,
    images,
    asin,
    parent_asin,
    user_id,
    DATE(TIMESTAMP_MILLIS(CAST(timestamp AS INT64))) AS review_date,
    CAST(helpful_vote AS INT64) AS helpful_vote,
    CAST(verified_purchase AS BOOL) AS verified_purchase,
    details,
    videos,
    _data_source,
    _load_time
FROM `amazon-product-reviews-452322.product_data_raw.reviews_Appliances`;

-- Create staging table for Automotive reviews
CREATE OR REPLACE TABLE `amazon-product-reviews-452322.product_data_stg.reviews_Automotive` AS
SELECT
    CAST(rating AS FLOAT64) AS rating,
    title,
    text,
    images,
    asin,
    parent_asin,
    user_id,
    DATE(TIMESTAMP_MILLIS(CAST(timestamp AS INT64))) AS review_date,
    CAST(helpful_vote AS INT64) AS helpful_vote,
    CAST(verified_purchase AS BOOL) AS verified_purchase,
    details,
    videos,
    _data_source,
    _load_time
FROM `amazon-product-reviews-452322.product_data_raw.reviews_Automotive`;

-- Create staging table for Books reviews
CREATE OR REPLACE TABLE `amazon-product-reviews-452322.product_data_stg.reviews_Books` AS
SELECT
    CAST(rating AS FLOAT64) AS rating,
    title,
    text,
    images,
    asin,
    parent_asin,
    user_id,
    DATE(TIMESTAMP_MILLIS(CAST(timestamp AS INT64))) AS review_date,
    CAST(helpful_vote AS INT64) AS helpful_vote,
    CAST(verified_purchase AS BOOL) AS verified_purchase,
    details,
    videos,
    _data_source,
    _load_time
FROM `amazon-product-reviews-452322.product_data_raw.reviews_Books`;

-- Create staging table for Digital Music reviews
CREATE OR REPLACE TABLE `amazon-product-reviews-452322.product_data_stg.reviews_Digital_Music` AS
SELECT
    CAST(rating AS FLOAT64) AS rating,
    title,
    text,
    images,
    asin,
    parent_asin,
    user_id,
    DATE(TIMESTAMP_MILLIS(CAST(timestamp AS INT64))) AS review_date,
    CAST(helpful_vote AS INT64) AS helpful_vote,
    CAST(verified_purchase AS BOOL) AS verified_purchase,
    details,
    videos,
    _data_source,
    _load_time
FROM `amazon-product-reviews-452322.product_data_raw.reviews_Digital_Music`;

-- Create staging table for Gift Cards reviews
CREATE OR REPLACE TABLE `amazon-product-reviews-452322.product_data_stg.reviews_Gift_Cards` AS
SELECT
    CAST(rating AS FLOAT64) AS rating,
    title,
    text,
    images,
    asin,
    parent_asin,
    user_id,
    DATE(TIMESTAMP_MILLIS(CAST(timestamp AS INT64))) AS review_date,
    CAST(helpful_vote AS INT64) AS helpful_vote,
    CAST(verified_purchase AS BOOL) AS verified_purchase,
    details,
    videos,
    _data_source,
    _load_time
FROM `amazon-product-reviews-452322.product_data_raw.reviews_Gift_Cards`;

-- Create staging table for Magazine Subscriptions reviews
CREATE OR REPLACE TABLE `amazon-product-reviews-452322.product_data_stg.reviews_Magazine_Subscriptions` AS
SELECT
    CAST(rating AS FLOAT64) AS rating,
    title,
    text,
    images,
    asin,
    parent_asin,
    user_id,
    DATE(TIMESTAMP_MILLIS(CAST(timestamp AS INT64))) AS review_date,
    CAST(helpful_vote AS INT64) AS helpful_vote,
    CAST(verified_purchase AS BOOL) AS verified_purchase,
    details,
    videos,
    _data_source,
    _load_time
FROM `amazon-product-reviews-452322.product_data_raw.reviews_Magazine_Subscriptions`;

-- Create staging table for Movies and TV reviews
CREATE OR REPLACE TABLE `amazon-product-reviews-452322.product_data_stg.reviews_Movies_and_TV` AS
SELECT
    CAST(rating AS FLOAT64) AS rating,
    title,
    text,
    images,
    asin,
    parent_asin,
    user_id,
    DATE(TIMESTAMP_MILLIS(CAST(timestamp AS INT64))) AS review_date,
    CAST(helpful_vote AS INT64) AS helpful_vote,
    CAST(verified_purchase AS BOOL) AS verified_purchase,
    details,
    videos,
    _data_source,
    _load_time
FROM `amazon-product-reviews-452322.product_data_raw.reviews_Movies_and_TV`;



Query is running:   0%|          |

""


In [116]:
%%bigquery --project amazon-product-reviews-452322

-- Create combined reviews table
CREATE OR REPLACE TABLE `amazon-product-reviews-452322.product_data_stg.product_reviews` AS
SELECT * FROM `amazon-product-reviews-452322.product_data_stg.reviews_Appliances`
UNION ALL
SELECT * FROM `amazon-product-reviews-452322.product_data_stg.reviews_Automotive`
UNION ALL
SELECT * FROM `amazon-product-reviews-452322.product_data_stg.reviews_Books`
UNION ALL
SELECT * FROM `amazon-product-reviews-452322.product_data_stg.reviews_Digital_Music`
UNION ALL
SELECT * FROM `amazon-product-reviews-452322.product_data_stg.reviews_Gift_Cards`
UNION ALL
SELECT * FROM `amazon-product-reviews-452322.product_data_stg.reviews_Magazine_Subscriptions`
UNION ALL
SELECT * FROM `amazon-product-reviews-452322.product_data_stg.reviews_Movies_and_TV`;

Query is running:   0%|          |

""


In [144]:
%%bigquery --project amazon-product-reviews-452322
DROP TABLE IF EXISTS `amazon-product-reviews-452322.product_data_stg.reviews_Appliances`;
DROP TABLE IF EXISTS `amazon-product-reviews-452322.product_data_stg.reviews_Automotive`;
DROP TABLE IF EXISTS `amazon-product-reviews-452322.product_data_stg.reviews_Books`;
DROP TABLE IF EXISTS `amazon-product-reviews-452322.product_data_stg.reviews_Digital_Music`;
DROP TABLE IF EXISTS `amazon-product-reviews-452322.product_data_stg.reviews_Gift_Cards`;
DROP TABLE IF EXISTS `amazon-product-reviews-452322.product_data_stg.reviews_Magazine_Subscriptions`;
DROP TABLE IF EXISTS `amazon-product-reviews-452322.product_data_stg.reviews_Movies_and_TV`;

Query is running:   0%|          |

""


# Requirement 7

Here we are splitting the list of categories out because they belong in their own col for easier access.




## User Table
Then loading users

In [139]:
%%bigquery --project amazon-product-reviews-452322
SELECT
    *
FROM product_data_raw.user
LIMIT 5;


Query is running:   0%|          |

Downloading:   0%|          |

,product_id,product_name,category,discounted_price,actual_price,discount_percentage,rating,rating_count,about_product,user_id,user_name,review_id,review_title,review_content,img_link,product_link,_data_source,_load_time
0,B0BPJBTB3F,Khaitan ORFin Fan heater for Home and kitchen-K0 2215,"Home&Kitchen|Heating,Cooling&AirQuality|RoomHeaters|FanHeaters","₹1,299","₹2,495",48%,2.0,2,Khaitan Orfin Fan heater for Home and kitchen|POWERFUL 2000 WATT|HEATING POSITION 1000 W-2000W|ADJUSTABLE THERMOSTAT TEMP.CONTROL|AUTOMATIC THERMAL CUTOUT FOR SAFETY|FRONT GRILL FOR SAFETY|TURBO FAN,"AGHT3K4KSG5MAQUSXRDT5VNB73GA,AE4Q5XQ7SZW35EEUJKQ3IV2IIBQQ","Manidipa Sengupta,Vidisha","R1OO2ED6615EX1,RR4S5JTJMCPA5","Bad quality,Amazing product..","The heating capacity is zero .Moreover i have initiated return request. Noone has come to collect it,Best heater at this price. Quality is very good . Suggest everyone to purchase this heater.......... amazing product to buy...",https://m.media-amazon.com/images/W/WEBP_402378-T1/images/I/31B8Pd1SmLL._SX300_SY300_QL70_FMwebp_.jpg,https://www.amazon.in/Khaitan-ORFin-heater-Home-kitchen-K0/dp/B0BPJBTB3F/ref=sr_1_326?qid=1672923609&s=kitchen&sr=1-326,sales,2025-03-01 23:28:18.289316+00:00
1,B0BFBNXS94,"Personal Size Blender, Portable Blender, Battery Powered USB Blender, with Four Blades, Mini Blender Travel Bottle for Juice, Shakes, and Smoothies (Pink)",Home&Kitchen|Kitchen&HomeAppliances|SmallKitchenAppliances|HandBlenders,₹669,"₹1,499",55%,2.3,13,"✔【Easy to carry around】- This handheld blender is equipped with a travel cover for easy carrying. You can drink nutritious juices, milkshakes or smoothies wherever you want, such as home, office, gym, travel or any other outdoor activities. In addition, it can be taken on the plane.|✔【Portable design】: The unique design can separate the base and the cup body. The USB juicer cup is equipped with 1400mAh rechargeable high-quality batteries that can be recharged in a variety of ways, like charging by power bank, AC Adapter, and charging time is 3-5 hours; fully charged can produce 15-20 cups of juice.|✔【food grade material】- The portable blender is made by high-quality ABS and silicone, food-grade material. It has unique safety design including Silicone bottom, non-slip and shock absorption.This portable juicer is also a suitable gift for juice and travel enthusiasts.|✔【4 Blade design】- The portable blender for milkshakes and smoothies has a powerful motor base and 4 food-grade stainless steel 3D blades.The SUS304 Stainless Stell of cutter head made with food-grade electrolysis technology is durable and has excellent mixing ability, allowing the pulp to be quickl|✔【One button blending/cleaning】: simple button touch. 350ml capacity when cleaning, just put an appropriate amount of water in the cup and press the button to automatically clean. This can save you a lot of trouble.","AHRDA66XO63XYCBZJMW4EUJN3BFQ,AELE2SOO7LBNHXU7UK5F7TGQHA6Q,AHAVCLRCPYO2MFYPTURF33N7XH5A,AE762UDUDQPW4R4QHHTIL7TPTJUA,AEGZSJIUSKF2EKIKGLNKY2CU6WXA","Manya,Been,Podili Parasuram,Rejoy Jacob,Priya","R1WJ8T3U9P42IU,RM9RH8FX9U95D,R31M8UXT7NLOMY,R18Q7M2R00EW68,R11NHZQ8OKA9U0","Nice product i recommend to buy,Do not buy!! Defective product,Wast of money,Very very bad portable,Waste of money its not working properly","I liked that it is so convenient to carry,Waste of money. Defective product, cheap quality. doesn’t blend at all,https://m.media-amazon.com/images/I/71IVsjyZ13L._SY88.jpg,First charge problemSecond motor proble,https://m.media-amazon.com/images/I/61aXXxIxPwL._SY88.jpg",https://m.media-amazon.com/images/I/417TQs3uroL._SX300_SY300_QL70_FMwebp_.jpg,https://www.amazon.in/Personal-Blender-Portable-Battery-Smoothies/dp/B0BFBNXS94/ref=sr_1_261_mod_primary_new?qid=1672923605&s=kitchen&sbo=RZvfv%2F%2FHxDF%2BO5021pAnSA%3D%3D&sr=1-261,sales,2025-03-01 23:28:18.289316+00:00
2,B0B7L86YCB,Green Tales Heat Seal Mini Food Sealer-Impulse Machine for Sealing Plastic Bags Packaging,Home&Kitchen|Kitchen&HomeAppl

In [142]:
%%bigquery --project amazon-product-reviews-452322
CREATE OR REPLACE TABLE `amazon-product-reviews-452322.product_data_stg.user` AS
SELECT
    product_id,
    product_name,
    CAST(REGEXP_REPLACE(CAST(discounted_price AS STRING), r'[^\d.]', '') AS FLOAT64) AS discounted_price,
    CAST(REGEXP_REPLACE(CAST(actual_price AS STRING), r'[^\d.]', '') AS FLOAT64) AS actual_price,
    SAFE_DIVIDE(CAST(REGEXP_REPLACE(CAST(discount_percentage AS STRING), r'[^\d]', '') AS FLOAT64), 100) AS discount_percentage,
    CAST(rating AS FLOAT64) AS rating,
    CAST(rating_count AS INT64) AS rating_count,
    NULLIF(about_product, '') AS about_product,
    SPLIT(CAST(user_id AS STRING), ',') AS user_id,
    SPLIT(CAST(user_name AS STRING), ',') AS user_name,
    SPLIT(CAST(review_id AS STRING), ',') AS review_id,
    SPLIT(CAST(review_title AS STRING), ',') AS review_title,
    SPLIT(CAST(review_content AS STRING), ',') AS review_content,
    img_link,
    product_link,
    _data_source,
    _load_time,
    SPLIT(CAST(category AS STRING), '|')[SAFE_OFFSET(0)] AS main_category,
    SPLIT(CAST(category AS STRING), '|')[SAFE_OFFSET(1)] AS subcategory_1,
    SPLIT(CAST(category AS STRING), '|')[SAFE_OFFSET(2)] AS subcategory_2,
    SPLIT(CAST(category AS STRING), '|')[SAFE_OFFSET(3)] AS subcategory_3,
    SPLIT(CAST(category AS STRING), '|')[SAFE_OFFSET(4)] AS subcategory_4,
    SPLIT(CAST(category AS STRING), '|')[SAFE_OFFSET(5)] AS subcategory_5,
    SPLIT(CAST(category AS STRING), '|')[SAFE_OFFSET(6)] AS subcategory_6,
    SPLIT(CAST(category AS STRING), '|')[SAFE_OFFSET(7)] AS subcategory_7,
    SPLIT(CAST(category AS STRING), '|')[SAFE_OFFSET(8)] AS subcategory_8
FROM `amazon-product-reviews-452322.product_data_raw.user`;


Query is running:   0%|          |

""


In [137]:
%%bigquery --project amazon-product-reviews-452322
SELECT column_name, data_type
FROM `amazon-product-reviews-452322.product_data_stg.INFORMATION_SCHEMA.COLUMNS`
WHERE table_name = 'user';


Query is running:   0%|          |

Downloading:   0%|          |

,column_name,data_type
0,product_id,STRING
1,product_name,STRING
2,category,STRING
3,discounted_price,FLOAT64
4,actual_price,FLOAT64
5,discount_percentage,FLOAT64
6,rating,FLOAT64
7,rating_count,INT64
8,about_product,STRING
9,user_id,ARRAY<STRING>


In [141]:
%%bigquery --project amazon-product-reviews-452322
SELECT COUNT(*) AS total_rows
FROM `amazon-product-reviews-452322.product_data_stg.user`;


Query is running:   0%|          |

Downloading:   0%|          |

,total_rows
0,1464


# Adding the rest of the ERD tables:
-  product_meta_data_2


In [147]:
%%bigquery --project amazon-product-reviews-452322
SELECT
    *
FROM product_data_raw.product_meta_data_2
LIMIT 5;


Query is running:   0%|          |

Downloading:   0%|          |

,Uniq Id,Product Name,Brand Name,Asin,Category,Upc Ean Code,List Price,Selling Price,Quantity,Model Number,About Product,Product Specification,Technical Details,Shipping Weight,Product Dimensions,Image,Variants,Sku,Product Url,Stock,Product Details,Dimensions,Color,_data_source,_load_time
0,dba7978b1f2986ecc9fe441b39e5c620,"Beistle CN028 30 & Stars Confetti Birthday Party Supplies, Tableware Decorations, 0.5 Ounces, Multicolored",None,None,Toys & Games | Party Supplies,696749891117 022735180284,NaN,$4.51,<NA>,CN028,The package contains 0. 5 oz. of confetti | Made of metallic plastic | Birthday party confetti consists of the number 30 and small Stars | Multicolor and cute | Spread on a table to decorate for the 30th birthday party,Product Dimensions: 5.2 x 4.5 inches ; 0.48 ounces |Shipping Weight: 0.2 ounces (View shipping rates and policies)|Domestic Shipping: Item can be shipped within U.S.|International Shipping: This item can be shipped to select countries outside of the U.S. Learn More|ASIN: B006UGXMNA|UPC: 696749891117 022735180284|Item model number: CN028| #51482 in Kids' Party Supplies,"Package Quantity:1 Beistle CN028 ""30"" And stars confetti. This confetti has assorted colors and is a mix of the Number 30 and stars. There is a half Ounce in the package. Spread this birthday confetti on tables to celebrate the person who is turning 30 This year!",0.2 ounces,5.2 x 4.5 inches 0.48 ounces,https://images-na.ssl-images-amazon.com/images/I/616qCUvVg1L.jpg|https://images-na.ssl-images-amazon.com/images/I/41rZZGR3m1L.jpg|https://images-na.ssl-images-amazon.com/images/G/01/x-locale/common/transparent-pixel.jpg,https://www.amazon.com/Beistle-CN028-30-Stars-Confetti/dp/B006UGXMNA|https://www.amazon.com/Beistle-CN028-30-Stars-Confetti/dp/B0128SPNPS|https://www.amazon.com/Beistle-CN028-30-Stars-Confetti/dp/B0128SPQC8|https://www.amazon.com/Beistle-CN028-30-Stars-Confetti/dp/B0128SEX40|https://www.amazon.com/Beistle-CN028-30-Stars-Confetti/dp/B0128SN30U|https://www.amazon.com/Beistle-CN028-30-Stars-Confetti/dp/B0128SNGHK|https://www.amazon.com/Beistle-CN028-30-Stars-Confetti/dp/B0128SOE9O,None,https://www.amazon.com/Beistle-CN028-30-Stars-Confetti/dp/B006UGXMNA,None,None,None,None,initial_loads/product_meta_data_2,2025-03-01 23:28:09.694386+00:00
1,3d33ab2be8c8366c2e156ad715e4cc3d,"Burton & Burton 260Q Neon Green Pencil Balloons, Pack Of 100",None,None,Toys & Games | Party Supplies | Balloons,071444764117 735533033354,NaN,$10.53,<NA>,76411.0,Latex | Made in USA | Enter Your Model Number | Above To Make Sure This Fits | Great For Parties And Other Special Occasions | Fill With Air | Qualatex - The Very Best Balloons,Product Dimensions: 60 x 2 x 2 inches ; 0.32 ounces |Shipping Weight: 0.3 ounces (View shipping rates and policies)|ASIN: B0077SY2ZM|UPC: 071444764117 735533033354|Item model number: 76411.0| #11549 in Kids' Party Balloons,Color:Neon Green When you're looking for a brand that stands out from the rest look no further. With a passion for quality and creativity you can rest easy knowing the quality we manufacture.,0.3 ounces,60 x 2 x 2 inches 0.32 ounces,https://images-na.ssl-images-amazon.com/images/I/21n68ul%2Bp2L.jpg|https://images-na.ssl-images-amazon.com/images/I/31c3P5JnIrL.jpg|https://images-na.ssl-images-amazon.com/images/G/01/x-locale/common/transparent-pixel.jpg,,None,https://www.amazon.com/Burton-260Q-Green-Pencil-Balloons/dp/B0077SY2ZM,None,None,None,None,initial_loads/product_meta_data_2,2025-03-01 23:28:09.694386+00:00
2,63f33269bd408e3cfa627e90785a6560,Beistle CN038 Tropical Breeze Confetti,None,None,,022735180383,NaN,$4.99,<NA>,CN038,This item is a great value! | 1/2 ounce per package | Luau party item | Confetti for festive occasions | High Quality,Product Dimensions: 5.2 x 4.5 inches ; 0.32 ounces |Shipping Weight: 0.3 ounces (View shipping rates and policies)|Domestic Shipping: Item can be shipped within U.S.|International Shipping: This item can be shipped to select countries outside of the U.S. Learn M

In [158]:
%%bigquery --project amazon-product-reviews-452322

-- Drop rows where Selling Price is 'Total price:' or has price ranges
CREATE OR REPLACE TABLE `amazon-product-reviews-452322.product_data_stg.product_meta_data_2` AS
SELECT *
FROM `amazon-product-reviews-452322.product_data_raw.product_meta_data_2`
WHERE LOWER(CAST(`Selling Price` AS STRING)) != 'total price:'
AND NOT REGEXP_CONTAINS(CAST(`List Price` AS STRING), r'\$\d+(\.\d+)?\s*-\s*\$\d+(\.\d+)?')
AND NOT REGEXP_CONTAINS(CAST(`Selling Price` AS STRING), r'\$\d+(\.\d+)?\s*-\s*\$\d+(\.\d+)?');


Query is running:   0%|          |

""


In [ ]:
%%bigquery --project amazon-product-reviews-452322
CREATE OR REPLACE TABLE `amazon-product-reviews-452322.product_data_stg.product_meta_data_2` AS
SELECT
    `Uniq Id` AS product_id,
    `Product Name` AS product_name,
    `Brand Name` AS brand_name,
    Asin AS asin,
    REGEXP_REPLACE(CAST(`Category` AS STRING), r'\|', ', ') AS category,
    `Upc Ean Code` AS upc_ean_code,
    CAST(NULLIF(REGEXP_REPLACE(CAST(`List Price` AS STRING), r'[$,]', ''), '') AS FLOAT64) AS list_price,
    CAST(NULLIF(REGEXP_REPLACE(CAST(`Selling Price` AS STRING), r'[$,]', ''), '') AS FLOAT64) AS selling_price,
    CAST(NULLIF(CAST(Quantity AS STRING), '') AS INT64) AS quantity,
    `Model Number` AS model_number,
    NULLIF(`About Product`, '') AS about_product,
    `Product Specification` AS product_specification,
    `Technical Details` AS technical_details,
    NULLIF(REGEXP_REPLACE(CAST(`Shipping Weight` AS STRING), r'[^\d.]', ''), '') AS shipping_weight,
    `Product Dimensions` AS product_dimensions,
    SPLIT(NULLIF(Image, ''), '|') AS image_urls,
    SPLIT(NULLIF(`Product Url`, ''), '|') AS product_urls,
    NULLIF(Variants, '') AS variants,
    NULLIF(Sku, '') AS sku,
    NULLIF(Stock, '') AS stock,
    `Product Details` AS product_details,
    NULLIF(Dimensions, '') AS dimensions,
    NULLIF(Color, '') AS color,
    _data_source,
    _load_time,
    SPLIT(CAST(`Category` AS STRING), '|')[SAFE_OFFSET(0)] AS main_category,
    SPLIT(CAST(`Category` AS STRING), '|')[SAFE_OFFSET(1)] AS subcategory_1,
    SPLIT(CAST(`Category` AS STRING), '|')[SAFE_OFFSET(2)] AS subcategory_2,
    SPLIT(CAST(`Category` AS STRING), '|')[SAFE_OFFSET(3)] AS subcategory_3,
    SPLIT(CAST(`Category` AS STRING), '|')[SAFE_OFFSET(4)] AS subcategory_4,
    SPLIT(CAST(`Category` AS STRING), '|')[SAFE_OFFSET(5)] AS subcategory_5,
    SPLIT(CAST(`Category` AS STRING), '|')[SAFE_OFFSET(6)] AS subcategory_6,
    SPLIT(CAST(`Category` AS STRING), '|')[SAFE_OFFSET(7)] AS subcategory_7,
    SPLIT(CAST(`Category` AS STRING), '|')[SAFE_OFFSET(8)] AS subcategory_8
FROM `amazon-product-reviews-452322.product_data_stg.product_meta_data_2`;


Query is running:   0%|          |

""


## Categories

In [ ]:
%%bigquery --project amazon-product-reviews-452322
SELECT * FROM `amazon-product-reviews-452322.product_data_raw.categories`
LIMIT 10;


Query is running:   0%|          |

Downloading:   0%|          |

,main_category,subcategory_1,subcategory_2,subcategory_3,subcategory_4,subcategory_5,subcategory_6,subcategory_7,subcategory_8,_data_source,_load_time
0,Appliances,None,None,None,None,None,None,None,None,categories,2025-03-01 23:25:50.547812+00:00
1,Appliances,Dishwashers,None,None,None,None,None,None,None,categories,2025-03-01 23:25:50.547812+00:00
2,Appliances,Dishwashers,Built-In Dishwashers,None,None,None,None,None,None,categories,2025-03-01 23:25:50.547812+00:00
3,Appliances,Dishwashers,Countertop Dishwashers,None,None,None,None,None,None,categories,2025-03-01 23:25:50.547812+00:00
4,Appliances,Dishwashers,Portable Dishwashers,None,None,None,None,None,None,categories,2025-03-01 23:25:50.547812+00:00
5,Appliances,Laundry Appliances,None,None,None,None,None,None,None,categories,2025-03-01 23:25:50.547812+00:00
6,Appliances,Laundry Appliances,Washers & Dryers,None,None,None,None,None,None,categories,2025-03-01 23:25:50.547812+00:00
7,Appliances,Laundry Appliances,Washers & Dryers,Clothes Dryers,None,None,None,None,None,categories,2025-03-01 23:25:50.547812+00:00
8,Appliances,Laundry Appliances,Washers & Dryers,Clothes Washing Machines,None,None,None,None,None,categories,2025-03-01 23:25:50.547812+00:00
9,Appliances,Laundry Appliances,Washers & Dryers,Combination Washers & Dryers,None,None,None,None,None,categories,2025-03-01 23:25:50.547812+00:00


In [ ]:
%%bigquery --project amazon-product-reviews-452322
CREATE OR REPLACE TABLE `amazon-product-reviews-452322.product_data_stg.categories` AS
SELECT
    main_category,
    subcategory_1,
    subcategory_2,
    subcategory_3,
    subcategory_4,
    subcategory_5,
    subcategory_6,
    subcategory_7,
    subcategory_8,
    _data_source,
    _load_time
FROM `product-data-448814.product_data_raw.categories`;


Query is running:   0%|          |

""


## Junction Table: user_reviews

In [175]:
%%bigquery --project amazon-product-reviews-452322
CREATE TABLE amazon-product-reviews-452322.product_data_stg.user_reviews (
    user_id STRING NOT NULL,
    review_id STRING NOT NULL
);


Executing query with job ID: c6b5e32d-f4e4-4bca-ba64-24b9818dede7
Query executing: 0.23s


ERROR:
 409 Already Exists: Table amazon-product-reviews-452322:product_data_stg.user_reviews; reason: duplicate, message: Already Exists: Table amazon-product-reviews-452322:product_data_stg.user_reviews

Location: us-central1
Job ID: c6b5e32d-f4e4-4bca-ba64-24b9818dede7



## product-image-data-all


In [176]:
%%bigquery --project amazon-product-reviews-452322

CREATE OR REPLACE TABLE `amazon-product-reviews-452322.product_data_stg.product_image_data_stg` AS
SELECT
    product_id,
    brand_name,
    product_name,
    asin,
    upc,
    main_category,
    subcategory_1,
    subcategory_2,
    subcategory_3,
    list_price,
    selling_price,
    quantity,
    model_number,
    about_product,
    product_specs,
    product_technical,
    weight,
    dimensions,
    url,
    detected_text,
    dominant_colors,
    object_features,
    image_path,
    _data_source,
    _load_time
FROM `amazon-product-reviews-452322.product_data_raw.product-image-data-all`;

Query is running:   0%|          |

""
